In [35]:
# !pip install pandas
# !pip install pinecone
# !pip unstall openai
# !pip install python-dotenv

In [36]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

True

In [37]:
token= os.getenv("GROQ_API_KEY") 
open_ai_base_url = os.getenv("EMBEDDING_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [38]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

## Try out embeddings

In [39]:
import torch
from sentence_transformers import SentenceTransformer

# 1. Load a free, high-quality embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Generate embeddings (returns a list of floats, just like you wanted)
#    Note: We use .tolist() to convert numpy array to a standard python list
embeddings = embed_model.encode("hello there").tolist()

print(len(embeddings)) # Should be 384 dimensions for this model
print(embeddings)

384
[-0.09443899989128113, 0.07322292774915695, 0.042268745601177216, 0.05430742725729942, -0.06719714403152466, -0.08545088022947311, 0.05023470148444176, 0.05221303552389145, -0.033030472695827484, -0.019267376512289047, 0.019890684634447098, -0.018779344856739044, -0.04508376121520996, -0.023084357380867004, -0.008017785847187042, -0.0005377819179557264, 0.01777532696723938, -0.007733933161944151, -0.1338099092245102, 0.0024418162647634745, -0.033949512988328934, 0.06793273985385895, -0.08245822787284851, 0.051667749881744385, -0.04819174483418465, -0.06251210719347, 0.031222917139530182, 0.021649783477187157, -0.009581751190125942, -0.04121425375342369, 0.014142850413918495, 0.0440271832048893, 0.040608130395412445, 0.04402764514088631, 0.04674624279141426, 0.0566849559545517, -0.09738067537546158, -0.059672459959983826, 0.05624128133058548, -0.010554807260632515, 0.01875927671790123, -0.049062661826610565, 0.0016443687491118908, -0.014013598673045635, 0.047891415655612946, -0.0865

In [40]:
len(embeddings)

384

## Wrangle dataset

In [41]:
df=pd.read_json('products/products.jsonl',lines=True)

In [42]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [43]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [44]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [45]:
texts = df['text'].tolist()

In [46]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [47]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Generate Embeddings

In [48]:
# --- REPLACE ALL OF THIS ---
# output = client.embeddings.create(...)
# ---------------------------

# --- WITH THIS LOCAL CODE ---
from sentence_transformers import SentenceTransformer

# 1. Load the specific Embedding Model (not Llama!)
#    This runs locally on your machine.
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Generate the embeddings locally
#    We use .tolist() to convert the result into a standard Python list,
#    which matches the format your previous code likely expected.
embeddings = embedding_model.encode(texts).tolist()

print(f"Generated {len(embeddings)} embeddings.")
print(embeddings[0]) # Uncomment to see the first vector

Generated 20 embeddings.
[0.033525433391332626, -0.05789956450462341, -0.0017869053408503532, 0.05488213524222374, 0.05808446183800697, -0.00026044162223115563, 0.01678725890815258, 0.042207907885313034, -0.022465791553258896, -0.0008332108845934272, -0.008280172944068909, -0.09297716617584229, -0.014481283724308014, -0.01984693482518196, -0.015685440972447395, -0.07400845736265182, 0.026576513424515724, -0.0028237923979759216, 0.0970730111002922, 0.020237214863300323, 0.006772992666810751, -0.0552884042263031, 0.03207572549581528, 0.10568635910749435, 0.1077440157532692, 0.09208441525697708, 0.050184231251478195, 0.025340666994452477, -0.0462079793214798, -0.07177939265966415, -0.052197039127349854, -0.06340939551591873, 0.06747890263795853, -0.028087209910154343, -0.06899981945753098, 0.08348885923624039, 0.10179390758275986, -0.04042471945285797, 0.05786528065800667, 0.0008215743582695723, -0.010293988510966301, 0.03710704296827316, 0.09358833730220795, 0.021180538460612297, 0.07276

## Push data to database

In [50]:
import time

# 1. List all existing indexes
existing_indexes = pc.list_indexes().names()

# 2. Check if your index is already in the list
if index_name not in existing_indexes:
    print(f"Creating index: {index_name}...")
    pc.create_index(
        name=index_name,
        dimension=384,     # Correct spelling (singular)
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    
    # Wait for the index to be ready before proceeding
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    print("Index created successfully!")
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")

# 3. Connect to the index (works in both cases)
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

Index 'mainproject1' already exists. Skipping creation.
Connected to index: mainproject1


In [53]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [55]:
# Assuming 'embed_model' is already defined (e.g., SentenceTransformer("all-MiniLM-L6-v2"))
input_text = ["Is Cappuccino lactose-free?"]

# Encode the text using the local model
output = embed_model.encode(input_text)

# The output is a numpy array. We extract the first embedding and convert it to a list.
embedding = output[0].tolist() 

print(f"Generated embedding vector of length: {len(embedding)}")

Generated embedding vector of length: 384


In [57]:
results = index.query(
    namespace="ns1",
    vector=embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.633574545,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 